# Simple Deep Fake
### First Order Motion Model for Image Animation



**Mount the Google drive folder on Colab**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

**Clone repository and download model**

In [ ]:
%%capture
!pip install PyYAML==5.3.1
!git clone https://github.com/AliaksandrSiarohin/first-order-model
!gdown https://drive.google.com/uc?id=1hWuhL87mIcbiM4iwQlcU3CBbQkRcRi_p
!gdown https://drive.google.com/uc?id=12nNacFcEBW37bqyPk4un5QXiqNPEFtyS

In [3]:
cd first-order-model

/content/drive/My Drive/Colab Notebooks/first-order-model


In [4]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

**Load driving video and source image**

In [5]:
source_image = imageio.imread('../src/face/note.png')
driving_video = imageio.mimread('../src/template-video/no-audio/sek_loso_template_no_audio.mp4', memtest=False, fps=10)

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

HTML(display(source_image, driving_video).to_html5_video())

**Load Model**

In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='../vox-cpk.pth.tar')

In [8]:
from demo import make_animation
from skimage import img_as_ubyte

**Relative True & Adaptive Movement False**

In [10]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 120/120 [00:15<00:00,  7.68it/s]


**Relative False & Adaptive Movement True**

In [11]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 120/120 [00:15<00:00,  7.71it/s]


**Relative True & Adaptive Movement True**

In [9]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 120/120 [00:15<00:00,  7.69it/s]


In [ ]:
imageio.mimsave('../src/output.mp4', [img_as_ubyte(frame) for frame in predictions])